In [1]:
!pip install optuna -q
!pip install transformers[torch] accelerate -q
!pip install pandas scikit-learn seaborn matplotlib -q

In [2]:
import pandas as pd
import torch
import numpy as np
import optuna
import os
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from transformers import (
    DistilBertTokenizer, RobertaTokenizer, XLNetTokenizer,
    DistilBertForSequenceClassification, RobertaForSequenceClassification, XLNetForSequenceClassification,
    Trainer, TrainingArguments, DataCollatorWithPadding
)
from torch.utils.data import Dataset
from optuna import Trial

MODEL_NAMES = ["distilbert", "roberta", "xlnet"]
N_OPTUNA_TRIALS = 5
N_TRAIN_EPOCHS = 3
OUTPUT_DIR = './comparative_results'

def load_and_preprocess_data():
    print("Loading and preprocessing data...")
    if not os.path.exists('spam_ham_dataset.csv'):
        raise FileNotFoundError("Error: 'spam_ham_dataset.csv' not found. Please ensure you have uploaded it to the Colab session.")

    df = pd.read_csv('spam_ham_dataset.csv')[['text', 'label']].dropna().sample(frac=1, random_state=42)
    X = df['text']
    y = df['label'].map({'ham': 0, 'spam': 1})

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print("Data loaded and split successfully.")
    return X_train, X_test, y_train, y_test

class SpamDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

def get_model_and_tokenizer(model_name):
    model_map = {
        "distilbert": ("distilbert-base-uncased", DistilBertForSequenceClassification, DistilBertTokenizer),
        "roberta": ("roberta-base", RobertaForSequenceClassification, RobertaTokenizer),
        "xlnet": ("xlnet-base-cased", XLNetForSequenceClassification, XLNetTokenizer)
    }
    pretrained_name, model_class, tokenizer_class = model_map[model_name]

    model = model_class.from_pretrained(pretrained_name, num_labels=2)
    tokenizer = tokenizer_class.from_pretrained(pretrained_name)

    return model, tokenizer

def main():
    X_train, X_test, y_train, y_test = load_and_preprocess_data()
    all_results = []

    for model_name in MODEL_NAMES:
        print(f"\n{'='*20} Processing Model: {model_name.upper()} {'='*20}")

        _, tokenizer = get_model_and_tokenizer(model_name)

        print(f"Tokenizing data for {model_name}...")
        train_encodings = tokenizer(list(X_train), truncation=True, padding=False, max_length=128)
        test_encodings = tokenizer(list(X_test), truncation=True, padding=False, max_length=128)

        train_dataset = SpamDataset(train_encodings, y_train.tolist())
        test_dataset = SpamDataset(test_encodings, y_test.tolist())
        data_collator = DataCollatorWithPadding(tokenizer)

        print("Starting hyperparameter tuning...")

        def objective(trial: Trial):
            learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-4, log=True)
            batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])

            model_for_trial, _ = get_model_and_tokenizer(model_name)

            training_args = TrainingArguments(
                output_dir=f"{OUTPUT_DIR}/{model_name}_optuna",
                num_train_epochs=1,
                per_device_train_batch_size=batch_size,
                learning_rate=learning_rate,
                logging_dir=f'./logs/{model_name}_optuna',
                eval_strategy="steps", # CORRECTED
                eval_steps=200,
                save_steps=200,
                save_strategy="steps",
                load_best_model_at_end=True,
                disable_tqdm=True,
                report_to="none"
            )

            trainer = Trainer(
                model=model_for_trial,
                args=training_args,
                train_dataset=train_dataset,
                eval_dataset=test_dataset,
                data_collator=data_collator,
            )

            trainer.train()
            eval_result = trainer.evaluate()
            return eval_result['eval_loss']

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=N_OPTUNA_TRIALS)
        best_params = study.best_trial.params
        print(f"Best hyperparameters found for {model_name}: {best_params}")

        print(f"Starting final training for {model_name} with best parameters...")
        final_model, _ = get_model_and_tokenizer(model_name)

        final_training_args = TrainingArguments(
            output_dir=f"{OUTPUT_DIR}/{model_name}_final",
            num_train_epochs=N_TRAIN_EPOCHS,
            per_device_train_batch_size=best_params['batch_size'],
            learning_rate=best_params['learning_rate'],
            weight_decay=0.01,
            warmup_steps=500,
            fp16=torch.cuda.is_available(),
            logging_dir=f'./logs/{model_name}_final',
            eval_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            report_to="none"
        )

        final_trainer = Trainer(
            model=final_model,
            args=final_training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
            data_collator=data_collator,
        )

        final_trainer.train()

        print(f"Evaluating final {model_name} model...")
        predictions = final_trainer.predict(test_dataset)
        y_pred = np.argmax(predictions.predictions, axis=-1)
        y_true = y_test.to_numpy()

        report = classification_report(y_true, y_pred, output_dict=True, target_names=['ham', 'spam'])
        roc_auc = roc_auc_score(y_true, y_pred)

        results = {
            "Model": model_name.capitalize(),
            "Accuracy": report['accuracy'],
            "Precision (Spam)": report['spam']['precision'],
            "Recall (Spam)": report['spam']['recall'],
            "F1-Score (Spam)": report['spam']['f1-score'],
            "ROC-AUC Score": roc_auc
        }
        all_results.append(results)

        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
        plt.title(f"Confusion Matrix - {model_name.capitalize()}")
        plt.xlabel("Predicted Label")
        plt.ylabel("True Label")
        plt.tight_layout()
        plt.savefig(f"{OUTPUT_DIR}/{model_name}_confusion_matrix.png")
        plt.close()
        print(f"Confusion matrix for {model_name} saved.")

    print("\n\n" + "="*25 + " FINAL COMPARATIVE RESULTS " + "="*25)
    results_df = pd.DataFrame(all_results)
    print(results_df.to_string(index=False))

if __name__ == "__main__":
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
    main()

Loading and preprocessing data...
Data loaded and split successfully.

==================== Processing Model: DISTILBERT ====================


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

{'eval_loss': 0.058007605373859406, 'eval_runtime': 4.3538, 'eval_samples_per_second': 237.723, 'eval_steps_per_second': 29.859, 'epoch': 1.0}
Best hyperparameters found for xlnet: {'learning_rate': 3.1002348124822405e-05, 'batch_size': 8}
Starting final training for xlnet with best parameters...


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.288000,0.138605
2,0.109400,0.086651
3,0.020300,0.110011


Evaluating final xlnet model...


Confusion matrix for xlnet saved.


========================= FINAL COMPARATIVE RESULTS =========================
     Model  Accuracy  Precision (Spam)  Recall (Spam)  F1-Score (Spam)  ROC-AUC Score
Distilbert  0.991304          0.986799       0.983553         0.985173       0.989040
   Roberta  0.989372          0.983498       0.980263         0.981878       0.986712
     Xlnet  0.986473          0.993197       0.960526         0.976589       0.978895


In [3]:
!zip -r /content/results.zip /content/comparative_results

  adding: content/comparative_results/ (stored 0%)
  adding: content/comparative_results/xlnet_optuna/ (stored 0%)
  adding: content/comparative_results/xlnet_optuna/checkpoint-259/ (stored 0%)
  adding: content/comparative_results/xlnet_optuna/checkpoint-259/tokenizer_config.json (deflated 81%)
  adding: content/comparative_results/xlnet_optuna/checkpoint-259/config.json (deflated 52%)
  adding: content/comparative_results/xlnet_optuna/checkpoint-259/training_args.bin (deflated 52%)
  adding: content/comparative_results/xlnet_optuna/checkpoint-259/special_tokens_map.json (deflated 52%)
  adding: content/comparative_results/xlnet_optuna/checkpoint-259/scaler.pt (deflated 60%)
  adding: content/comparative_results/xlnet_optuna/checkpoint-259/spiece.model (deflated 49%)
  adding: content/comparative_results/xlnet_optuna/checkpoint-259/scheduler.pt (deflated 55%)
  adding: content/comparative_results/xlnet_optuna/checkpoint-259/optimizer.pt (deflated 21%)
  adding: content/comparative_res